<a href="https://colab.research.google.com/github/miladmirzazadeh/Gamein-Prototype/blob/master/prototype_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TESTING CODES

In [ ]:
Controller.create_player("milad")

In [98]:
Controller.create_player("hasan")

In [118]:
Controller.player_objects["hasan"].land_objects 
for i in Controller.player_objects.keys():
  for land in Controller.player_objects[i].land_objects.values():
    print(i,land, land.machine_objects.keys())

milad <__main__.Land object at 0x7f7741c377b8> dict_keys([2])
hasan <__main__.Land object at 0x7f7741c37160> dict_keys([2])


END   TESTING CODES





In [77]:
import numpy as np 
import pandas as pd 
import random

In [87]:
class Controller():
  return_message=""
  turn = 0 
  last_turn = 50
  player_objects = dict()#name,object

  def next_turn():
    turn+=1
    if turn == Controller.last_turn : 
      score_board = Controller.score_board()
      Controller.return_message = "End Game. Thank u " + score_board

  
  def score_board():
    net_worths = pd.Series([])
    for player in list(player_objects.keys()):
      obj = player_objects[player]
      obj.calculate_final_net_worth()
      net_worths[player] = obj 
    net_worths = net_worths.sort(ascending=False)
    return net_worths





  # def get_status(player_name):
    # for land in player_objecs[player_name].


  def create_player(player_name):
    land_id = random.choice(Land.available_lands) #random از بین لیست زمین های در دسترس
    Land.available_lands.remove(land_id)   #remove from available lands

    machine_id =  2 # از دیتافریم زمین و ماشین خوانده میشود
 
    Controller.player_objects.update({player_name : Player(player_name)})
    Controller.buy_land(player_name, land_id)
    Controller.buy_machine(player_name, land_id, machine_id)
    
  def buy_land(player_name, land_id):
    Controller.player_objects[player_name].add_land(land_id)
    # Controller.player_objects[name].cash -=         اینجا باید پول زمین رو از حسابش کم کنیم . فعلا لازم نیست  چون خودمون بهش زمین رو اختصاص میدیم 
    
  def buy_machine(player_name, land_id, machine_id):
    Controller.player_objects[player_name].add_machine(land_id, machine_id)
    
  def put_into_market(player_name, product, amount, price):
    player_obj = Controller.player_objects[player_name]
    stockhouse_obj = _
    product_obj = _
    Market.products_on_vitrin_objects.update({product_on_vitrin_id : ProductOnVitrin(player_obj, stockhouse_obj, product_obj, amount, price))})
    pass
  
  def upgrade_stockhouse(player_name, land_id):
    pass

  def show_product_market_info():
    return Market.products_on_vitrin

  def buy_product(player_name, product_on_vitrin_id, amount): # آیدی محصول داخل ویترین رو میده که بریم از روی دیکشنری توی مارکت آبجکتش رو پیدا کنیم و انتقال رو انجام بدیم 
    cash = Controller.player_objects[player_name].cash 
    obj = Market.product_on_vitrin_objects[product_on_vitrin_id]
    if cash < obj.price * amount :
      Controller.return_message = "u have not enough cash"
      break 
    else :
      obj.amount -= 
   

In [57]:
x = pd.DataFrame([1,2,3])

In [96]:
class Player():
  players_name=[]

  def __init__(self, player_name):
    self.name = player_name
    #remove from available
    self.cash = 1000000 # بودجه اولیه
    self.land_objects=dict()

  def calculate_final_net_worth(self):
    inventory_value_list = []
    for land in list(self.land_objects.keys()):
      inventory_value_list.append(self.land_objects[land].calculate_inventory_value()) # تو هر خونه این لیست ، ارزش دارایی های یک زمین از زمین های بازیکن ثبت میشه
    inventory_value_sum = sum(inventory_value_list)
    net_worth = inventory_value_sum + self.cash 
    return net_worth 

  def add_land(self, land_id):
    self.land_objects.update({land_id : Land(land_id)}) # یه اینستنس از لند میسازه که ورودی بهش آیدی میدیم . بعد توی خود لند ، بر حسب آیدی بقیه فیچر های لند رو ست میکنیم
    

  def add_machine(self, land_id, machine_id):
    self.land_objects[land_id].add_machine(machine_id)







In [ ]:
class Gamein_Bazaar():

  def put_into_market():
    

In [67]:
ali = Player("ali")

In [70]:
ali.add_land(_)


TypeError: ignored

In [69]:
ali.land_objects

{22: <__main__.Land at 0x7f7741c2f6d8>}

In [108]:
class Land():
  lands_list =[] #amirreza یه دیتا فریم که یه آی دی زمین داره و ماشین
  available_lands=[1,2,3,4,5,6] #آی دی های باقی مانده
  machine_objects = dict()
  def __init__(self, land_id):
    self.land_id = land_id
    # self.stockhouse = StockHouse()
  def add_machine(self, machine_id):
    self.machine_objects.update({machine_id: Machine(machine_id)})

    


  

In [109]:
class StockHouse():
  def __init__(self):
    self.remaining_cap = 3000
    self.inventory_df = pd.DataFrame()
    self.cost_per_product = 100

  

In [94]:
class Machine():
  machines_list = _

  def __init__(self,machine_id):
    self.production_id = _
    self.price = _ 
    self.enabled = True
    self.cost_per_production = _
    self.working = 0
    self.remaining_production = _
    
  def start_production(self):
    pass
  def stop_production(self):
    pass
  

  

In [29]:
class Product():
  def __init__(self):
    self.material_needed = _
    self.lower_band = _
    self.upper_band = _

    
  

In [18]:
class Material():
  Prices = _ #dict (id , price)

In [30]:
class Market():
  product_on_vitrin_objects = dict()
def __init__(self, player_obj, product_obj, amount, price):
  self.player_name = player_obj.name 
  
  products_on_vitrin = pd.DataFrame([], columns=["Supplier", "Product", "Amount", "Price", "Storehouse_object"])
def add_to_market(player_name,):
  pass
def remove_from_market():
  pass 

In [119]:
class ProductOnVitrin():
  def __init__(self, player_obj, stockhouse_obj, product_obj, amount, price):
    

SyntaxError: ignored

In [ ]:
ali = Player("aliagha",1)
milad = Player("mili",1)

In [ ]:
Player.players_name.append(ali.name)
Player.players_name.append(milad.name)

In [ ]:
ali.calculate_net_worth()

UnboundLocalError: ignored

In [ ]:
class a():
  def x_2(): 
    print("hi")
  def y():
    a.x_2()